In [ ]:
# 我自己服务器账号用不了，借gsm账号创建文件；

In [1]:
from odps import ODPS
from odps.inter import setup, enter, teardown
%load_ext odps
teardown()
setup('LTAIa5twar4KoCfW',
      'gegwRbj9zRPbWHKy1bCoEgI7zV59Kw',
      'cashbus_etl',
      'http://service.odps.aliyun.com/api')
room = enter()
o = room.odps

1. 根据created 选这一个月 所有loanid; 

In [11]:
%sql create table qb_halfyear_label_1 LIFECYCLE 3 as select loanid, userid, loanstyle, funddate, duedate, payofftime, default_now from dw_gcswhj_defaults_raw where funddate < '2019-07-17 16:43:09' and funddate > '2019-01-17 16:43:09'


TransientProgressBar(value=0.0)

InstancesProgress()

HTMLNotifier()

''

In [12]:
%sql create table qb_halfyear_label_2 LIFECYCLE 3 as select loanid, userid, loanstyle, funddate, duedate, payofftime, default_now from dw_cardfq_installment_defaults_raw where funddate < '2019-07-17 16:43:09' and funddate > '2019-01-17 16:43:09'


TransientProgressBar(value=0.0)

InstancesProgress()

HTMLNotifier()

''

In [13]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/'
i = 'loan_label_1_1901_1907'
sql_expr1 = "select * from qb_halfyear_label_1"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

start
-- use 14.207358121871948 seconds


In [14]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/'
i = 'loan_label_2_1901_1907'
sql_expr1 = "select * from qb_halfyear_label_2"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

start
-- use 302.51447916030884 seconds


In [15]:
col_dir = '/data-0/gsm/qb_one_month/'
i = 'loan_label_2_1901_1907'
path = col_dir + i + '.csv'

In [16]:
import pandas as pd

In [ ]:
# 分期产品，每笔只取做决策的那一行；

In [18]:
lable2 = pd.read_csv(path)
lable2['row_num'] = range(lable2.shape[0])

In [23]:
groups = lable2.groupby('loanid')
row_no = []
excep = []
count = 0
for i,j in groups:
    count+=1
    if count%10000 == 0:
        print('group num:{}'.format(count))
    j = j.sort_values('duedate')
    num_install = j.shape[0]
    if num_install==1:  # 单期用户没啥歧异；
        val = j.row_num.values
    elif num_install == 2:  # 两期用户只要还一期，就约等于好用户, 只拿第一期那一行；
        val = j.row_num.values[0]
    elif num_install == 3:  # 三期用户 还两期，约等于好用户；
        val = j.row_num.values[1]
    elif num_install == 5:  # 5-3
        val = j.row_num.values[2]
    elif num_install == 12: # 12-6
        val = j.row_num.values[5]
    else:
        excep.append(i)
    row_no.append(val)

group num:10000
group num:20000
group num:30000
group num:40000
group num:50000
group num:60000
group num:70000
group num:80000
group num:90000
group num:100000
group num:110000
group num:120000
group num:130000
group num:140000
group num:150000
group num:160000
group num:170000
group num:180000
group num:190000
group num:200000
group num:210000
group num:220000
group num:230000
group num:240000
group num:250000
group num:260000
group num:270000
group num:280000
group num:290000
group num:300000
group num:310000
group num:320000
group num:330000
group num:340000
group num:350000
group num:360000
group num:370000
group num:380000
group num:390000
group num:400000
group num:410000
group num:420000
group num:430000
group num:440000
group num:450000
group num:460000
group num:470000
group num:480000
group num:490000
group num:500000
group num:510000
group num:520000
group num:530000
group num:540000
group num:550000
group num:560000
group num:570000
group num:580000
group num:590000
group 

In [26]:
to_save_df = lable2[lable2.row_num.isin(row_no)]

In [32]:
to_save_df.to_csv(col_dir + 'loan_label_2_new_1901_1907.csv', header=None,index=False)

In [36]:
to_save_df.head()

,loanid,userid,loanstyle,funddate,duedate,payofftime,default_now,row_num
1,85536700,13014742,红卡3月3期,2019-01-18 00:00:00,2019-03-18 00:00:00,2019-03-31 16:18:10,0.0,1
4,85753319,13371995,绿卡3月3期,2019-01-18 00:00:00,2019-03-18 00:00:00,NaN,1.0,4
7,85836810,811093,绿卡3月3期,2019-01-20 00:00:00,2019-03-20 00:00:00,2019-05-09 17:18:32,0.0,7
9,85857057,15649622,绿卡30天2期,2019-01-20 00:00:00,2019-02-04 00:00:00,2019-02-18 19:04:21,0.0,9
12,85878265,5550606,红卡3月3期,2019-01-18 00:00:00,2019-03-18 00:00:00,NaN,0.0,12


2. 根据上面的loanid 找到所有userid, 然后查询相应属性；

若根据半年内created loanid 找userid, 则有的userid 可能出现时间及其早； 
～15种关系实体 是什么时间的实体？能保证每个关系实体都有时间属性吗？

如果根据userid updated 在半年内， 则完美覆盖半年内 created loan； 但是多余出来两类:
1. 部分user 只更新 信息，但是没有被funded；
2. 有些user 两年前的loan 在最近半年还清了，也会被记录；



In [3]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/'
i = 'qb_geohash_halfyear_1901_1907_test_10'
sql_expr1 = "select * from qb_geohash_halfyear_1901_1907 limit 10;"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + '/' + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

start


ConnectionError: HTTPConnectionPool(host='service.odps.aliyun.com', port=80): Max retries exceeded with url: /api/projects/cashbus_etl/instances?curr_project=cashbus_etl (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd43fb422e8>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [3]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/'
i = 'mobile_device_update_1901_1907'
sql_expr1 = "SELECT created, updated, userid, deviceid, lastlogintime, androidid, imei1, imei1real, mac, pushregid, idfv, idfa from mobile_device where updated < '2019-07-17 16:43:09' and updated > '2019-01-17 16:43:09';"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + '/' + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

start
-- use 344.65618348121643 seconds


In [5]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/phoneRecord_halfYear'
i = 'phoneRecord_1906_1907'
sql_expr1 = "select * from qb_phonerecord_halfyear_1901_1907 where create_date < '20190717' and create_date > '20190617';"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + '/' + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

start
-- use 3605.7666041851044 seconds


In [2]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/phoneRecord_halfYear'
i = 'phoneRecord_1905_1906'
sql_expr1 = "select * from qb_phonerecord_halfyear_1901_1907 where create_date < '20190617' and create_date > '20190517';"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + '/' + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

start


ConnectionError: HTTPConnectionPool(host='service.odps.aliyun.com', port=80): Max retries exceeded with url: /api/projects/cashbus_etl/instances?curr_project=cashbus_etl (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd4444c86a0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [ ]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/phoneRecord_halfYear'
i = 'phoneRecord_1904_1905'
sql_expr1 = "select * from qb_phonerecord_halfyear_1901_1907 where create_date < '20190517' and create_date > '20190417';"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + '/' + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

In [ ]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/phoneRecord_halfYear'
i = 'phoneRecord_1903_1904'
sql_expr1 = "select * from qb_phonerecord_halfyear_1901_1907 where create_date < '20190417' and create_date > '20190317';"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + '/' + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

In [ ]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/phoneRecord_halfYear'
i = 'phoneRecord_1902_1903'
sql_expr1 = "select * from qb_phonerecord_halfyear_1901_1907 where create_date < '20190317' and create_date > '20190217';"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + '/' + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))

In [ ]:
import time
print('start')
st = time.time()
col_dir = '/data-0/gsm/qb_one_month/phoneRecord_halfYear'
i = 'phoneRecord_1901_1902'
sql_expr1 = "select * from qb_phonerecord_halfyear_1901_1907 where create_date < '20190217' and create_date > '20190117';"
with o.execute_sql(sql_expr1).open_reader(limit_enabled=False) as reader:
    df = reader.to_pandas()
    df.to_csv(col_dir + '/' + i + '.csv', index = False)
    print('-- use {} seconds'.format(time.time() - st))